In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import mean_squared_error
import mlflow

# Import Statsmodels
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.vector_ar.vecm import coint_johansen

# misc
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Read the wide data 
all_wide_data = pd.read_csv('df_final_date_wide_2007.csv')
all_wide_data.shape

(65363, 40)

### Utility Functions

In [3]:
# Function to add the column to the main data
def add_column_to_main_data(data, cluster_data, col_name):
    
    cluster_data['ags5'] = cluster_data['ags5'].apply(fix_ags5)
    
    # Get the cluster and ags 5 and set ags 5 as index 
    cluster_info = cluster_data.set_index('ags5').to_dict()[col_name]
    
    data['cluster'] = '0'
    data['cluster'] = data['ags5'].map(cluster_info)

    # check if the cluster have been allotted correctly 
    print("original cluster data")
    print(cluster_data[col_name].value_counts())
    print("New data")
    print(data.drop_duplicates(subset=['ags5'])['cluster'].value_counts())
    
    return data

In [4]:
# Fix ags 5
def fix_ags5(x):
    """Function to fix the format of ag5 """
    if len(str(x))==4:
        return '0'+str(x)
    else:
        return str(x)
all_wide_data['ags5'] = all_wide_data['ags5'].apply(fix_ags5)

In [5]:
all_wide_data['ags5']

0        01001
1        01001
2        01001
3        01001
4        01001
         ...  
65358    16077
65359    16077
65360    16077
65361    16077
65362    16077
Name: ags5, Length: 65363, dtype: object

## Create ML Flow Evaluation function

In [6]:
def evaluate_var_model(all_data, params):
    
    # Create a runname
    run_name = 'var_' + str(len(params['relevant_cols'])) + 'cols_' + str(params['lag_value']) + 'lag_diff_' + str(params['second_diff'])
    
    ''' Utility Functions '''
    
    # Invert the transformation to get the real forecast
    def invert_transformation(df_train, df_forecast, second_diff=False):
        
        """Revert back the differencing to get the forecast to original scale."""
        df_fc = df_forecast.copy()
        columns = df_train.columns
        for col in columns:        
            # Roll back 2nd Diff
            if second_diff:
                df_fc[str(col)+'_diff'] = (df_train[col].iloc[-1]-df_train[col].iloc[-2]) + df_fc[str(col)+'_diff'].cumsum()
            # Roll back 1st Diff
            df_fc[str(col)+'_forecast'] = df_train[col].iloc[-1] + df_fc[str(col)+'_diff'].cumsum()
        return df_fc
    
    # Start the MLflow experiment 
    with mlflow.start_run(run_name=run_name):
        
        # Get list of kreis 
        kreis_list = list(all_data['ags5'].unique())
        
        # Define total metrics 
        total_mse, total_rmse, total_mape = 0, 0, 0
        
        print("Fitting all models", end='')
        # Iterate through each kreis
        for kreis in kreis_list:
            
            print('.', end='')
            
            # Filter data by kreis
            kreis_data = all_wide_data[all_wide_data['ags5'] == kreis]
            
            # Convert date format
            kreis_data['date'] = pd.to_datetime(kreis_data['date'])
            
            # Clip the data before 2010 and after 2019 
            start_date = '2010-01-01'
            end_date = '2020-01-01'
            mask = (kreis_data['date'] >= start_date) & (kreis_data['date'] < end_date)
            useful_data = kreis_data[(mask)]
            
            # Set the date as the index
            useful_data.set_index('date', inplace=True)
            
            # Select the relevant columns from the data
            relevant_cols = params['relevant_cols']
            useful_data = useful_data[relevant_cols]
            
            ''' Modelling Fitting'''
            
            # Split the training and testing data 
            n_obs = params['num_predictions']
            df_train, df_test = useful_data[0:-n_obs], useful_data[-n_obs:]
            
            # Differentiate the data twice if asked else once
            if_second_diff = params['second_diff']
        
            df_differenced = df_train.diff().dropna()
            if if_second_diff:
                df_differenced = df_differenced.diff().dropna()
            
            # Get the lag value 
            lag_value = params['lag_value']
            
            # Feed the model data
            model = VAR(df_differenced)
            
            # Fit the model to lag
            model_fitted = model.fit(lag_value)
            
            '''Forecasting section'''
            # Get the lag order
            lag_order = model_fitted.k_ar

            # Input data for forecasting
            forecast_input = df_differenced.values[-lag_order:]
            
            # Get the forecasts
            fc = model_fitted.forecast(y=forecast_input, steps=n_obs)
            df_forecast = pd.DataFrame(fc, index=useful_data.index[-n_obs:], columns=useful_data.columns + '_diff')
            
            # Invert the forecast results
            df_results = invert_transformation(df_train, df_forecast, second_diff=if_second_diff)
            final_results = df_results.filter(regex='_forecast')
            
            ''' Evaluation '''
            forecast = df_results['unemployment_rate_forecast'].values
            actual = df_test['unemployment_rate']
            mse = mean_squared_error(actual, forecast, squared=True)
            rmse = mean_squared_error(forecast, actual, squared=False)
            mape = np.mean(np.abs(forecast - actual)/np.abs(actual))
            
            # add to the totals
            total_mse += mse
            total_rmse += rmse
            total_mape += mape
            
        ''' Log metrics '''
        
        # Calculate errors
        average_mse = total_mse/len(kreis_list)
        average_rmse = total_rmse/len(kreis_list)
        average_mape = total_mape/len(kreis_list)
        
        # Log model 
#         mlflow.statsmodels.log_model(model, "var_model")
        
        # log params
        mlflow.log_params(params)
        
        # Log errors
        mlflow.log_metric("mse", average_mse)
        mlflow.log_metric("rmse", average_rmse)
        mlflow.log_metric("mape", average_mape)

            

In [7]:
relevant_cols = ['number_of_company_deletions', 'number_of_company_liquidations',
       'number_of_start_ups', 'unemployment_rate', 'registerd_jobs',
       'unemployed', 'employees_social_security_at_work',
       'employees_social_security_at_residence']

In [8]:
params = {
    'num_predictions' : 12,
    'lag_value': 4, 
    'relevant_cols': relevant_cols,
    'second_diff': False,
}

evaluate_var_model(all_wide_data, params)

Fitting all models.................................................................................................................................................................................................................................................................................................................................................................................................................

In [9]:
relevant_cols = ['number_of_company_deletions', 
                 'number_of_company_liquidations',
                 'number_of_start_ups',
                 'unemployment_rate',
#                  'registerd_jobs',
#                  'unemployed', 
#                  'employees_social_security_at_work',
#                  'employees_social_security_at_residence'
                ]

In [10]:
# intial params 
params = {
    'num_predictions' : 12,
    'lag_value': 4, 
    'relevant_cols': relevant_cols,
    'second_diff': True,
}

# Try a combination of lag and diff values 
lags = [i for i in range(1, 10)]
diff = [True, False]

for l in lags: 
    for d in diff:
        print(l, d)
        params['second_diff'] = d
        params['lag_value'] = l
    
        evaluate_var_model(all_wide_data, params)

1 True
Fitting all models.................................................................................................................................................................................................................................................................................................................................................................................................................1 False
Fitting all models.................................................................................................................................................................................................................................................................................................................................................................................................................2 True
Fitting all models..........................................................................................................................

KeyboardInterrupt: 

## VAR with clusters

In [35]:
def evaluate_var_model_with_clusters(all_data, cluster_col, params, cluster_type='ags2'):
    
    # Create a runname
    run_name = 'naive_cluster_var_' + str(len(params['relevant_cols'])) + 'cols_' + str(params['lag_value']) + 'lag_diff_' + str(params['second_diff'])
    
    ''' Utility Functions '''
    
    # Invert the transformation to get the real forecast
    def invert_transformation(df_train, df_forecast, second_diff=False):
        
        """Revert back the differencing to get the forecast to original scale."""
        df_fc = df_forecast.copy()
        columns = df_train.columns
        for col in columns:        
            # Roll back 2nd Diff
            if second_diff:
                df_fc[str(col)+'_diff'] = (df_train[col].iloc[-1]-df_train[col].iloc[-2]) + df_fc[str(col)+'_diff'].cumsum()
            # Roll back 1st Diff
            df_fc[str(col)+'_forecast'] = df_train[col].iloc[-1] + df_fc[str(col)+'_diff'].cumsum()
        return df_fc
    
    # Start the MLflow experiment 
    with mlflow.start_run(run_name=run_name):
        
        # Make a list of ags 2 
        cluster_list = list(all_data[cluster_col].unique())
        
        # Establish total error metrics
        total_mse, total_rmse, total_mape = 0, 0, 0
        count = 1
        # Iterate through the clusters 
        for cluster in cluster_list:
            print("Cluster: ", cluster)
        
            
            # Get all kreis in the clusters
            cluster_data = all_data[all_wide_data[cluster_col] == cluster]

            ''' Create a dataframe of the unemployment rate '''
            kreis_list = list(cluster_data['ags5'].unique())
            print(kreis_list)
            
            # Check if the kreis has a single item
            if len(kreis_list) == 1:
                print("Single item in the cluster. Hence skipping.")
                continue
            
            # Create the data 
            unemployment_rate_df = cluster_data[cluster_data['ags5']==kreis_list[0]][['date']].reset_index(drop=True) # start with the date column
            
            # Iterate through the kreis
            for kreis in kreis_list:
                filter_data = cluster_data[cluster_data['ags5'] == kreis].reset_index()
                col_name = kreis + '_unemployment_rate'
                unemployment_rate_df[col_name] = filter_data['unemployment_rate']

            # Convert date format 
            unemployment_rate_df['date'] = pd.to_datetime(unemployment_rate_df['date'])
            
            # Clip the data before 2010 and after 2019 
            start_date = '2010-01-01'
            end_date = '2020-01-01'
            mask = (unemployment_rate_df['date'] >= start_date) & (unemployment_rate_df['date'] < end_date)
            input_df = unemployment_rate_df[(mask)]

            input_df.set_index('date', inplace=True)
            
            ''' Modelling Section '''
            
            # Perform a train test split
            n_obs = params['num_predictions']
            df_train, df_test = input_df[0:-n_obs], input_df[-n_obs:]
            
            # Differentiate the data twice if asked else once
            if_second_diff = params['second_diff']

            df_differenced = df_train.diff().dropna()
            if if_second_diff:
                df_differenced = df_differenced.diff().dropna()
                
            # Get the lag value 
            lag_value = params['lag_value']

            # Feed the model data
            model = VAR(df_differenced)

            # Fit the model to lag
            model_fitted = model.fit(lag_value)
            
            '''Forecasting section'''
            # Get the lag order
            lag_order = model_fitted.k_ar

            # Input data for forecasting
            forecast_input = df_differenced.values[-lag_order:]

            # Get the forecasts            
            fc = model_fitted.forecast(y=forecast_input, steps=n_obs)
            df_forecast = pd.DataFrame(fc, index=input_df.index[-n_obs:], columns=input_df.columns + '_diff')

            # Invert the forecast results
            df_results = invert_transformation(df_train, df_forecast, second_diff=if_second_diff)
            final_results = df_results.filter(regex='_forecast')
            
            ''' Evaluation '''
            
            # Iterate through all the kreis in the results 
            for kreis in kreis_list:
                
                forecast = df_results[kreis + '_unemployment_rate_forecast'].values
                actual = df_test[kreis + '_unemployment_rate']
                                 
                mse = mean_squared_error(actual, forecast, squared=True)
                rmse = mean_squared_error(forecast, actual, squared=False)
                mape = np.mean(np.abs(forecast - actual)/np.abs(actual))

                # add to the totals
                total_mse += mse
                total_rmse += rmse
                total_mape += mape
                print("kreis evaluated:", count)
                count+=1

        ''' Log metrics '''
        count -= 1
        print(count)
        # Calculate errors
        average_mse = total_mse/count
        average_rmse = np.sqrt(average_mse)
        average_mape = total_mape/count

        # Log model 
#         mlflow.statsmodels.log_model(model, "var_model")

        # log params
        mlflow.log_params(params)
        mlflow.log_param("cluster_column", cluster_col)
        mlflow.log_param("cluster_type", cluster_type)
        
        # Log errors
        mlflow.log_metric("mse", average_mse)
        mlflow.log_metric("rmse", average_rmse)
        mlflow.log_metric("mape", average_mape)


In [12]:
relevant_cols = ['number_of_company_deletions', 
                 'number_of_company_liquidations',
                 'number_of_start_ups',
                 'unemployment_rate',
                 'registerd_jobs',
                 'unemployed', 
                 'employees_social_security_at_work',
                 'employees_social_security_at_residence'
                ]

In [34]:
# intial params 
params = {
    'num_predictions' : 12,
    'lag_value': 4, 
    'relevant_cols': relevant_cols,
    'second_diff': True,
}

# Try a combination of lag and diff values 
lags = [i for i in range(1, 10)]
diff = [True, False]

for l in lags: 
    for d in diff:
        print(l, d)
        params['second_diff'] = d
        params['lag_value'] = l
    
        evaluate_var_model_with_clusters(all_data=all_wide_data, params=params, cluster_type='bundesland', cluster_col='ags2')

1 True
Cluster:  1
['01001', '01002', '01003', '01004', '01051', '01053', '01054', '01055', '01056', '01057', '01058', '01059', '01060', '01061', '01062']
kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
Cluster:  2
['02000']
Single item in the cluster. Hence skipping.
Cluster:  3
['03101', '03102', '03103', '03151', '03153', '03154', '03155', '03157', '03158', '03159', '03241', '03251', '03252', '03254', '03255', '03256', '03257', '03351', '03352', '03353', '03354', '03355', '03356', '03357', '03358', '03359', '03360', '03361', '03401', '03402', '03403', '03404', '03405', '03451', '03452', '03453', '03454', '03455', '03456', '03457', '03458', '03459', '03460', '03461', '03462']
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis eval

kreis evaluated: 274
kreis evaluated: 275
kreis evaluated: 276
kreis evaluated: 277
kreis evaluated: 278
kreis evaluated: 279
kreis evaluated: 280
kreis evaluated: 281
kreis evaluated: 282
kreis evaluated: 283
kreis evaluated: 284
kreis evaluated: 285
kreis evaluated: 286
kreis evaluated: 287
kreis evaluated: 288
kreis evaluated: 289
kreis evaluated: 290
kreis evaluated: 291
kreis evaluated: 292
kreis evaluated: 293
kreis evaluated: 294
kreis evaluated: 295
kreis evaluated: 296
kreis evaluated: 297
kreis evaluated: 298
kreis evaluated: 299
kreis evaluated: 300
kreis evaluated: 301
kreis evaluated: 302
kreis evaluated: 303
kreis evaluated: 304
kreis evaluated: 305
kreis evaluated: 306
kreis evaluated: 307
kreis evaluated: 308
kreis evaluated: 309
kreis evaluated: 310
kreis evaluated: 311
kreis evaluated: 312
kreis evaluated: 313
kreis evaluated: 314
kreis evaluated: 315
kreis evaluated: 316
kreis evaluated: 317
Cluster:  10
['10041', '10042', '10043', '10044', '10045', '10046']
kreis ev

kreis evaluated: 178
kreis evaluated: 179
kreis evaluated: 180
kreis evaluated: 181
kreis evaluated: 182
kreis evaluated: 183
kreis evaluated: 184
kreis evaluated: 185
kreis evaluated: 186
kreis evaluated: 187
kreis evaluated: 188
kreis evaluated: 189
kreis evaluated: 190
kreis evaluated: 191
kreis evaluated: 192
kreis evaluated: 193
kreis evaluated: 194
kreis evaluated: 195
kreis evaluated: 196
kreis evaluated: 197
kreis evaluated: 198
kreis evaluated: 199
kreis evaluated: 200
kreis evaluated: 201
kreis evaluated: 202
kreis evaluated: 203
kreis evaluated: 204
kreis evaluated: 205
kreis evaluated: 206
kreis evaluated: 207
kreis evaluated: 208
kreis evaluated: 209
kreis evaluated: 210
kreis evaluated: 211
kreis evaluated: 212
kreis evaluated: 213
kreis evaluated: 214
kreis evaluated: 215
kreis evaluated: 216
kreis evaluated: 217
kreis evaluated: 218
kreis evaluated: 219
kreis evaluated: 220
kreis evaluated: 221
Cluster:  9
['09161', '09162', '09163', '09171', '09172', '09173', '09174', 

kreis evaluated: 63
kreis evaluated: 64
kreis evaluated: 65
kreis evaluated: 66
kreis evaluated: 67
kreis evaluated: 68
kreis evaluated: 69
kreis evaluated: 70
kreis evaluated: 71
kreis evaluated: 72
kreis evaluated: 73
kreis evaluated: 74
kreis evaluated: 75
kreis evaluated: 76
kreis evaluated: 77
kreis evaluated: 78
kreis evaluated: 79
kreis evaluated: 80
kreis evaluated: 81
kreis evaluated: 82
kreis evaluated: 83
kreis evaluated: 84
kreis evaluated: 85
kreis evaluated: 86
kreis evaluated: 87
kreis evaluated: 88
kreis evaluated: 89
kreis evaluated: 90
kreis evaluated: 91
kreis evaluated: 92
kreis evaluated: 93
kreis evaluated: 94
kreis evaluated: 95
kreis evaluated: 96
kreis evaluated: 97
kreis evaluated: 98
kreis evaluated: 99
kreis evaluated: 100
kreis evaluated: 101
kreis evaluated: 102
kreis evaluated: 103
kreis evaluated: 104
kreis evaluated: 105
kreis evaluated: 106
kreis evaluated: 107
kreis evaluated: 108
kreis evaluated: 109
kreis evaluated: 110
kreis evaluated: 111
kreis ev

kreis evaluated: 361
kreis evaluated: 362
Cluster:  15
['15001', '15002', '15003', '15081', '15082', '15083', '15084', '15085', '15086', '15087', '15088', '15089', '15090', '15091']
kreis evaluated: 363
kreis evaluated: 364
kreis evaluated: 365
kreis evaluated: 366
kreis evaluated: 367
kreis evaluated: 368
kreis evaluated: 369
kreis evaluated: 370
kreis evaluated: 371
kreis evaluated: 372
kreis evaluated: 373
kreis evaluated: 374
kreis evaluated: 375
kreis evaluated: 376
Cluster:  16
['16051', '16052', '16053', '16054', '16055', '16056', '16061', '16062', '16063', '16064', '16065', '16066', '16067', '16068', '16069', '16070', '16071', '16072', '16073', '16074', '16075', '16076', '16077']
kreis evaluated: 377
kreis evaluated: 378
kreis evaluated: 379
kreis evaluated: 380
kreis evaluated: 381
kreis evaluated: 382
kreis evaluated: 383
kreis evaluated: 384
kreis evaluated: 385
kreis evaluated: 386
kreis evaluated: 387
kreis evaluated: 388
kreis evaluated: 389
kreis evaluated: 390
kreis eva

...

kreis evaluated: 350
kreis evaluated: 351
kreis evaluated: 352
kreis evaluated: 353
kreis evaluated: 354
kreis evaluated: 355
kreis evaluated: 356
kreis evaluated: 357
kreis evaluated: 358
kreis evaluated: 359
kreis evaluated: 360
kreis evaluated: 361
kreis evaluated: 362
Cluster:  15
['15001', '15002', '15003', '15081', '15082', '15083', '15084', '15085', '15086', '15087', '15088', '15089', '15090', '15091']
kreis evaluated: 363
kreis evaluated: 364
kreis evaluated: 365
kreis evaluated: 366
kreis evaluated: 367
kreis evaluated: 368
kreis evaluated: 369
kreis evaluated: 370
kreis evaluated: 371
kreis evaluated: 372
kreis evaluated: 373
kreis evaluated: 374
kreis evaluated: 375
kreis evaluated: 376
Cluster:  16
['16051', '16052', '16053', '16054', '16055', '16056', '16061', '16062', '16063', '16064', '16065', '16066', '16067', '16068', '16069', '16070', '16071', '16072', '16073', '16074', '16075', '16076', '16077']
kreis evaluated: 377
kreis evaluated: 378
kreis evaluated: 379
kreis eva

kreis evaluated: 222
kreis evaluated: 223
kreis evaluated: 224
kreis evaluated: 225
kreis evaluated: 226
kreis evaluated: 227
kreis evaluated: 228
kreis evaluated: 229
kreis evaluated: 230
kreis evaluated: 231
kreis evaluated: 232
kreis evaluated: 233
kreis evaluated: 234
kreis evaluated: 235
kreis evaluated: 236
kreis evaluated: 237
kreis evaluated: 238
kreis evaluated: 239
kreis evaluated: 240
kreis evaluated: 241
kreis evaluated: 242
kreis evaluated: 243
kreis evaluated: 244
kreis evaluated: 245
kreis evaluated: 246
kreis evaluated: 247
kreis evaluated: 248
kreis evaluated: 249
kreis evaluated: 250
kreis evaluated: 251
kreis evaluated: 252
kreis evaluated: 253
kreis evaluated: 254
kreis evaluated: 255
kreis evaluated: 256
kreis evaluated: 257
kreis evaluated: 258
kreis evaluated: 259
kreis evaluated: 260
kreis evaluated: 261
kreis evaluated: 262
kreis evaluated: 263
kreis evaluated: 264
kreis evaluated: 265
kreis evaluated: 266
kreis evaluated: 267
kreis evaluated: 268
kreis evaluat

kreis evaluated: 116
kreis evaluated: 117
kreis evaluated: 118
kreis evaluated: 119
kreis evaluated: 120
kreis evaluated: 121
kreis evaluated: 122
kreis evaluated: 123
kreis evaluated: 124
kreis evaluated: 125
kreis evaluated: 126
kreis evaluated: 127
kreis evaluated: 128
kreis evaluated: 129
kreis evaluated: 130
kreis evaluated: 131
kreis evaluated: 132
kreis evaluated: 133
kreis evaluated: 134
kreis evaluated: 135
kreis evaluated: 136
kreis evaluated: 137
kreis evaluated: 138
kreis evaluated: 139
kreis evaluated: 140
kreis evaluated: 141
Cluster:  7
['07111', '07131', '07132', '07133', '07134', '07135', '07137', '07138', '07140', '07141', '07143', '07211', '07231', '07232', '07233', '07235', '07311', '07312', '07313', '07314', '07315', '07316', '07317', '07318', '07319', '07320', '07331', '07332', '07333', '07334', '07335', '07336', '07337', '07338', '07339', '07340']
kreis evaluated: 142
kreis evaluated: 143
kreis evaluated: 144
kreis evaluated: 145
kreis evaluated: 146
kreis evalua

kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
Cluster:  2
['02000']
Single item in the cluster. Hence skipping.
Cluster:  3
['03101', '03102', '03103', '03151', '03153', '03154', '03155', '03157', '03158', '03159', '03241', '03251', '03252', '03254', '03255', '03256', '03257', '03351', '03352', '03353', '03354', '03355', '03356', '03357', '03358', '03359', '03360', '03361', '03401', '03402', '03403', '03404', '03405', '03451', '03452', '03453', '03454', '03455', '03456', '03457', '03458', '03459', '03460', '03461', '03462']


KeyboardInterrupt: 

### Clusters using PCA

In [14]:
relevant_cols = ['number_of_company_deletions', 
                 'number_of_company_liquidations',
                 'number_of_start_ups',
                 'unemployment_rate',
                 'registerd_jobs',
                 'unemployed', 
                 'employees_social_security_at_work',
                 'employees_social_security_at_residence'
                ]

In [24]:
# read the pca clusters by Amit 
cluster1 = pd.read_csv('df_final_stationary.csv', converters={'ags5':str, 'cluster':str})
cluster1_input = add_column_to_main_data(all_wide_data, cluster1, 'cluster')

original cluster data
0    332
2     67
1      2
Name: cluster, dtype: int64
New data
0    332
2     67
1      2
Name: cluster, dtype: int64


In [22]:
# intial params 
params = {
    'num_predictions' : 12,
    'lag_value': 4, 
    'relevant_cols': relevant_cols,
    'second_diff': True,
}

# Try a combination of lag and diff values 
lags = [i for i in range(1, 10)]
diff = [True, False]

for l in lags: 
    for d in diff:
        print(l, d)
        params['second_diff'] = d
        params['lag_value'] = l
    
        evaluate_var_model_with_clusters(all_data=cluster1_input, params=params, cluster_type='pca clusters with correct count', cluster_col='cluster')

1 True


NameError: name 'cluster1_input' is not defined

### kmodes clusters

In [25]:
# Read the data 
cluster2 = pd.read_csv('kmodes3.csv', converters={'ags5':str, 'cluster':str})
print(cluster2.shape)
cluster2.head()

(401, 3)


,kreis,ags5,cluster
0,"Flensburg, Stadt",1001,1
1,"Kiel, Landeshauptstadt",1002,2
2,"Lübeck, Hansestadt",1003,2
3,"Neumünster, Stadt",1004,0
4,Dithmarschen,1051,1


In [53]:
cluster2_input = add_column_to_main_data(all_wide_data, cluster2, 'cluster')

original cluster data
2    190
0    114
1     97
Name: cluster, dtype: int64
New data
2    190
0    114
1     97
Name: cluster, dtype: int64


In [54]:
d = cluster2_input[cluster2_input['ags5'] == '16077']
d2 = cluster2_input[cluster2_input['ags5'] == '16077']
d2['ags5'] = "123"
cluster2_input = d2.append(d)

In [55]:
cluster2_input

,ags2,ags5,date,number_of_company_deletions,number_of_company_liquidations,number_of_start_ups,number_of_companies_administration,number_of_companies_agriculture,number_of_companies_arts_entertainment,number_of_companies_communication,...,realized_short_time_work_people,underemployment_without_short_time _work,unemployment_benefit_entitled,unemployment_benefit_recipients,unemployment_rate,registerd_jobs,unemployed,employees_social_security_at_work,employees_social_security_at_residence,cluster
56000,16,123,5/1/2007,3,1,6,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,18.1,245,9673,26462,33711,0
56001,16,123,6/1/2007,2,2,2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,17.3,253,9252,26564,33890,0
56002,16,123,7/1/2007,4,1,4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,17.1,269,9149,26687,34040,0
56003,16,123,8/1/2007,5,2,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,17.3,255,9254,26745,34064,0
56004,16,123,9/1/2007,1,2,4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,16.9,284,9023,27165,34537,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65358,16,16077,7/1/2020,2,1,5,114.0,100.0,122.0,44.0,...,2709.0,4505.0,6661.0,1022.0,7.2,664,3242,27356,33148,0
65359,16,16077,8/1/2020,1,1,4,114.0,100.0,121.0,44.0,...,2104.0,4432.0,6653.0,1024.0,7.3,692,3271,27490,33287,0
65360,16,16077,9/1/2020,1,0,5,114.0,100.0,121.0,44.0,...,1750.0,4277.0,6473.0,970.0,7.0,806,3129,27765,33583,0
65361,16,16077,10/1/2020,0,0,7,114.0,100.0,121.0,44.0,...,1553.0,4193.0,6326.0,946.0,6.8,774,3050,27866,33603,0


In [46]:
set(cluster2_input['ags5'])

{'01002', '123'}

In [56]:
# intial params 
params = {
    'num_predictions' : 3,
    'lag_value': 4, 
    'relevant_cols': relevant_cols,
    'second_diff': True,
}

# Try a combination of lag and diff values 
lags = [i for i in range(1, 2)]
diff = [False]

for l in lags: 
    for d in diff:
        print(l, d)
        params['second_diff'] = d
        params['lag_value'] = l
    
        evaluate_var_model_with_clusters(all_data=cluster2_input, params=params, cluster_type='kmodes clusters', cluster_col='cluster')

1 False
Cluster:  0
['123', '16077']
kreis evaluated: 1
kreis evaluated: 2
2


## tsne clusters


In [20]:
# Read the tsne data
cluster3 = pd.read_csv('df_tsne_cluster.csv', converters={'ags5':str, 'cluster':str})
print(cluster3.shape)
cluster3.head()

(401, 179)


,data_index,cluster,kreis,ags5,ags2,supermarkets_population,supermarkets_average_distance,public_transport_availability,average_distance_bus_stop,average_distance_train_station,...,room_type_location,district_settlement_structure,type_of_settlement_structure,urban_/_rural,metropolitan_region,metropolitan_area,east_west,border_proximity,support_area_status,eligible_area
0,0,2,"Flensburg, Stadt",1001,1,92,500,35,240,2901,...,2,4,3,2,99,99,1,1,C,1
1,1,2,"Kiel, Landeshauptstadt",1002,1,92,460,37,268,2037,...,2,1,2,1,99,99,1,0,C/D,1
2,2,2,"Lübeck, Hansestadt",1003,1,90,532,37,297,1927,...,1,1,2,1,5,99,1,0,C/D,1
3,3,2,"Neumünster, Stadt",1004,1,85,588,37,316,1648,...,2,3,2,2,5,99,1,0,D,1
4,4,0,Dithmarschen,1051,1,51,1864,35,448,3517,...,4,4,3,2,5,99,1,0,C,1


In [21]:
cluster3_input = add_column_to_main_data(all_wide_data, cluster3, 'cluster')

original cluster data
0    179
1    129
2     93
Name: cluster, dtype: int64
New data
0    179
1    129
2     93
Name: cluster, dtype: int64


In [22]:
# intial params 
params = {
    'num_predictions' : 12,
    'lag_value': 4, 
    'relevant_cols': relevant_cols,
    'second_diff': True,
}

# Try a combination of lag and diff values 
lags = [i for i in range(1, 10)]
diff = [True, False]

for l in lags: 
    for d in diff:
        print(l, d)
        params['second_diff'] = d
        params['lag_value'] = l
    
        evaluate_var_model_with_clusters(all_data=cluster3_input, params=params, cluster_type='tsne clusters', cluster_col='cluster')

1 True
Cluster:  2
kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis eval

1 False
Cluster:  2
kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis eva

Cluster:  2
kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis evaluated: 

2 False
Cluster:  2
kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis eva

3 True
Cluster:  2
kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis eval

...

kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis evaluated: 50
kreis eva

8 True
Cluster:  2
kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis eval

8 False
Cluster:  2
kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis eva

Cluster:  2
kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis evaluated: 

kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis evaluated: 50
kreis eva